# Sprint　論文読解入門

# (1) 物体検出の分野にはどういった手法が存在したか。

R-CNNやFast R-CNNが存在した。

R-CNN[2]では、物体候補領域検出 (Region Proposal) と分類 (CNN + SVM) を行うアーキテクチャが提案されました。
Region ProposalにはSelective Searchを採用してます。Selective Searchは類似したセグメント (最小単位は画素) をグルーピングしていくことで、複数のセグメントに分割する手法。
Selective Searchで得られた全ての領域に対して、CNNで特徴抽出し、SVMで分類するというものでした。

Fast R-CNNでは、R-CNNで領域候補の数だけCNNを通していた点に着目し、元の画像からCNNで得られた特徴マップを入力として分類する手法を提案しました。これによってCNNを通す回数が画像あたり1回になり、大幅に推論時間を短縮しました。
一方でRegion Proposalの計算は、R-CNN同様Selective Searchが使われていました。

その結果、Region Proposalの推論時間が大部分 (本論文の実験では80%以上) を占めるようになりました。動画に対するリアルタイムな物体検出 (5fps以上) を実現するためには、このRegion Proposalの短縮化が最後のハードルになってました。

### 参照

1 INTRODUCTION

Recent advances in object detection are driven by
the success of region proposal methods (e.g., [4])
and region-based convolutional neural networks (RCNNs) [5]. Although region-based CNNs were computationally expensive as originally developed in [5],
their cost has been drastically reduced thanks to sharing convolutions across proposals [1], [2]. The latest
incarnation, Fast R-CNN [2], achieves near real-time
rates using very deep networks [3], when ignoring the
time spent on region proposals. Now, proposals are the
test-time computational bottleneck in state-of-the-art
detection systems.

より

先行研究論文1
[2] R. Girshick, “Fast R-CNN,” in IEEE International Conference on
Computer Vision (ICCV), 2015.


先行研究論文2
https://arxiv.org/pdf/1504.08083.pdf

# (2) Fasterとあるが、どういった仕組みで高速化したのか。

・先発のFast R-CNNで時間を要していた物体領域候補の抽出を、ニューラルネットワークで置き換えたことで、推論を高速化

・物体の領域抽出処理と分類を1つのネットワークに統合することで、一気通貫で学習できるようになった

### 参照
3.1 Region Proposal Networksに乗っていると思います

# (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

One-Stage
YOLO（You Only Look Once）やSSD（Singe Shot MultiBox Detector）などのシングルステージディテクターは、入力画像を取得し、クラス確率と境界ボックス座標を学習することにより、オブジェクト検出を単純な回帰問題として扱います。このようなモデルは精度が低くなりますが、2ステージのオブジェクト検出器よりもはるかに高速です。

一段のCNNで直接boundingboxの座標とクラスを推定する


Two-Stage
（i）Region Proposal Networkを使用して第1段階で関心領域を生成し、 （ii）オブジェクトの分類と境界ボックス回帰のために、パイプラインを介して領域提案を送信します。このようなモデルは最高の精度率に達しますが、通常は低速です。

i)候補領域の抽出
ii)クラス、boundingboxの座標推定

### 参照論文

Optimizing the Trade-off between Single-Stage and Two-Stage Object Detectors using Image Difficulty Prediction
Petru Soviany, Radu Tudor Ionescu
(Submitted on 23 Mar 2018 (v1), last revised 31 Aug 2018 (this version, v3))

# (4) RPNとは何か。

Region Proposal Networkの略。RPNは、入力画像中から物体候補領域(物体が存在し得る画像領域の候補)を抽出するための小さな畳み込みネットワークです。RPNへ渡す特徴マップの生成前に一度だけCNNを実行します。Fast R-CNN以前の手法では、物体領域抽出にSelective Searchと呼ばれるディープラーニング以前の手法を使っていたため、抽出性能が低く候補が無数に抽出されてしまい、物体候補領域の抽出とその後の識別処理の計算コストが膨大となってしまっていました。

Faster R-CNNでは、物体候補領域の抽出をディープラーニングのモデルに取り込み、End-to-Endで学習・推定することで高性能な物体候補領域の抽出を実現し、その結果従来よりも高速かつ高性能な一般物体検出を実現しています。

RPNは、物体候補を出力するための2つの機能を持っています。1つ目は、図３中の赤枠内の画像が物体かどうかを表すスコアを計算する機能(図3中のcls layer)です。2つ目は、赤枠の概説矩形のスケールや位置を回帰により微調整する機能(図2中のreg layer)です。赤枠は、あらかじめ定義されたk個の外接矩形(Anchor)を用いて決定されます。このAnchor boxにさまざまな形、サイズを用意しておくとで多種多様な物体を検出できるようになるわけです。

# (5) RoIプーリングとは何か。

目的は、不均一なサイズの入力に対して最大プールを実行して、固定サイズの特徴マップ（たとえば7×7）を得ることです。
画像の多くの領域候補が必ず重複し、同じCNN計算を何度も(最大2000回)実行していました。なので、画像を一回のみCNN実行し、領域間で計算結果を共有しようと試みました。

CNNの特徴マップから対応する領域を選択することによって、各領域のCNN特徴がどのように取得されるかを確認します。次に各領域の特徴がプーリングされます（通常 max poolingを使用します）。したがって、従来手法の最大2000回とは対照的に、元の画像の1回のパスのみCNNが実行されます。

入力リストからすべての関心領域について、それに対応する入力特徴マップのセクションを取り、それを予め定義されたサイズ（例えば、7×7）にスケールします。スケーリングは次の方法で行います。

1.領域候補を同じサイズのセクションに分割します（その数は出力の次元と同じ）
2.各セクションで最大値を見つける
3.これらの最大値を出力バッファにコピーする
その結果、サイズの異なる長方形のリストから、固定サイズの対応する特徴マップのリストをすばやく取得できます。RoIプーリング出力の次元は、実際には入力特徴マップのサイズや領域提案のサイズに依存しないことに注意。領域候補を分割するセクションの数だけによって決定されます。

RoIプーリングのメリットの1つは処理速度です。フレームに複数の物体候補がある場合（通常はたくさんある）、それらのすべてに対して同じ入力特徴マップを使用できます。ネットワーク処理の初期段階での畳み込み計算は非常にコストがかかるので、このアプローチは時間を大幅に節約できます。



### 参照論文

Rich feature hierarchies for accurate object detection and semantic segmentation
https://arxiv.org/abs/1311.2524

# (6) Anchorのサイズはどうするのが適切か

サイズは目的による。さまざまな種類のアンカー/ボックスを自由に設計できます。たとえば、乗客/歩行者をカウントするネットワークを設計している場合、非常に短い、非常に大きい、または正方形のボックスを考慮する必要がない場合がある。きちんとしたアンカーセットを使用すると、速度と精度が向上する場合がある。

Anchor boxesは一つのAnchorに対して、例えば3（基準の長さ）×3（縦横比）=9つ作られます。
Anchor boxesを作る際には各基準の長さごとにAnchor boxesの面積は揃える必要があることには注意する。
つまり基準の長さ=64としたとき、Anchor boxesは
1:1→64×64 (=4096)
1:2→45×91 (≒4096)
2:1→91×45 (≒4096)
というサイズになります。尚、画像からはみ出たAnchor boxesは無視します。
この論文では20000から6000にアンカーを減らしている。

### 参照

P6

The anchor boxes that cross image boundaries need to be handled with care. During training, we ignore all cross-boundary anchors so they do not contribute to the loss. 
For a typical 1000 × 600 image, there will be roughly 20000 (≈ 60 × 40 × 9) anchors in total. 
With the cross-boundary anchors ignored, there are about 6000 anchors per image for training. 
If the boundary-crossing outliers are not ignored in training,
they introduce large, difficult to correct error terms in the objective, and training does not converge.
 During testing, however, we still apply the fully convolutional RPN to the entire image. 
This may generate crossboundary proposal boxes, which we clip to the image boundary.

# (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

PASCAL VOC 2007, 2012, そして MS COCO datasetsを使った。
指標値は mAP(mean Average Precision)を使っている。これは、全ての時点iにおけるAP(平均適合率)を平均することで、より一般的な平均値を求める為のものです。ある画像(物体)の情報が与えられた時点までの適合率(Precision)の平均であるAP(Average Precision)の平均

$
mAP = \frac{1}{M}\sum_{i=1}^M AP_i 
$

table2は、PASCAL VOC 2007テストセットの検出結果（VOC 2007 trainvalでトレーニング）。検出器は、ZFを使用した高速R-CNNですが、トレーニングとテストにさまざまな提案方法を使用しています。先行研究method(SS)ではmAP58,7%であったのに対し、method(RPN+ZF, shared)はmAP59,9%になっており、methodを変えたことで精度が上がっていることがわかる。

table5はproposal と detectionの両方にかかる時間について。rateが０.５FPSから５FPSになり速度が１０倍になったという指標が得られている。

table6ではPASCAL VOC 2007,PASCAL VOC 2012,COCOのデータ３種類を組み合わせた場合のmPA比較をしていて、どの場合も先行研究（model:SS)よりも(model:RPN)の方がmPAが高い。

### 参照

table2,teble5,table6

In Table 5 we summarize the running time of the
entire object detection system. SS takes 1-2 seconds
depending on content (on average about 1.5s), and
Fast R-CNN with VGG-16 takes 320ms on 2000 SS
proposals (or 223ms if using SVD on fully-connected
layers [2]). Our system with VGG-16 takes in total
198ms for both proposal and detection. With the convolutional features shared, the RPN alone only takes
10ms computing the additional layers. Our regionwise computation is also lower, thanks to fewer proposals (300 per image). Our system has a frame-rate
of 17 fps with the ZF net.